In [1]:
import pandas as pd
import csv

In [2]:
# Om te beginnen laden we de gecureerde versie van de WIC-opvarenden database in
opvarenden_curated = pd.read_excel('curated/script_output_curated.xlsx')

Tijdelijk: we verwijderen gesplitste data ('/') uit de dataset. 

In [3]:
# Filter alle rijen met '/' erin ...
columns_to_check = ['name', 'location', 'ship_name', 'imp_organization', 'captain', 'Bestemming', 'final_creditor_name', 'final_debt_amount_int']
rows_to_remove = opvarenden_curated[opvarenden_curated[columns_to_check].apply(lambda x: x.str.contains('/')).any(axis=1)]

# ... en verwijder die!
opvarenden_curated = opvarenden_curated.drop(rows_to_remove.index)

In [4]:
# Eerst proberen we de namen van schuldeisers te normaliseren aan de hand van een vooraf voorbereide lijst
normalized_creditors = {}
with open('curated/normalized_creditors.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter=';')
    next(reader) # header
    for row in reader:
        if row[1].strip():  # Check  of row[1] niet leeg is
            normalized_creditors[row[0]] = row[1]

opvarenden_curated['normalized_creditor_name'] = opvarenden_curated['final_creditor_name'].map(normalized_creditors)
opvarenden_curated['normalized_creditor_name'] = opvarenden_curated['normalized_creditor_name'].fillna(opvarenden_curated['final_creditor_name'])

In [5]:
# Nu hetzelfde voor de rollen
normalized_roles = {}
with open('curated/normalized_roles.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter=';')
    next(reader)  # header
    for row in reader:
        if row[1].strip() and row[2].strip():  
            normalized_roles[row[0]] = {'normalized_role': row[1], 'role_category': row[2]}

opvarenden_curated['normalized_role'] = opvarenden_curated['role'].map(lambda x: normalized_roles.get(x, {}).get('normalized_role', x))
opvarenden_curated['normalized_role_category'] = opvarenden_curated['role'].map(lambda x: normalized_roles.get(x, {}).get('role_category', ''))

In [6]:
# Nu hetzelfde voor schepen
normalized_ships = {}
with open('curated/normalized_ships.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter=';')
    next(reader) # header
    for row in reader:
        normalized_ships[row[0]] = row[1]

opvarenden_curated['normalized_ship_name'] = opvarenden_curated['ship_name'].map(normalized_ships)
opvarenden_curated['normalized_ship_name'] = opvarenden_curated['normalized_ship_name'].fillna(opvarenden_curated['ship_name'])

In [7]:
# Nu laden we de locaties in 
temp_locations = pd.read_excel('curated/normalized_locations.xlsx')

# Eerst maken we een lijst met alle unieke locaties
unique_locations = temp_locations[['place_standardized', 'country_code', 'geonames_uri', 'latitude', 'longitude']].drop_duplicates()

# er zitten nog wat NaNs in
unique_locations = unique_locations.dropna(subset=['place_standardized', 'geonames_uri'])

# We gebruiken betere namen voor de kolommen
Locations = unique_locations.rename(columns={
    'place_standardized': 'label',
    'country_code': 'country',
    'geonames_uri': 'geonames_uri',
    'latitude': 'latitude',
    'longitude': 'longitude'
})

# We geven iedere locatie een id
Locations.insert(0, 'location_id', range(1, 1 + len(Locations)))

# Dict om (label, geonames_uri) naar location_id te mappen
location_to_index = { (row['label'], row['geonames_uri']): row['location_id'] for _, row in Locations.iterrows() }

# wic_location geeft een link naar de locatie 
temp_locations['wic_location'] = temp_locations.apply(lambda row: location_to_index.get((row['place_standardized'], row['geonames_uri']), None) if pd.notna(row['place_standardized']) and pd.notna(row['geonames_uri']) else None, axis=1)

# Voor locaties zonder Geonames koppeling, maar wel een country_code
new_locations = pd.DataFrame([
    {'location_id': 2000, 'label': 'Frankrijk (land)', 'country': 'FR', 'geonames_uri': 'http://sws.geonames.org/3017382/', 'latitude': 46.603354, 'longitude': 1.888334},
    {'location_id': 2001, 'label': 'Ierland (land)', 'country': 'IE', 'geonames_uri': 'http://sws.geonames.org/2963597/', 'latitude': 53.41291, 'longitude': -8.24389},
    {'location_id': 2002, 'label': 'Groot-Brittannië (land)', 'country': 'GB', 'geonames_uri': 'http://sws.geonames.org/2635167/', 'latitude': 54.7023545, 'longitude': -3.2765753},
    {'location_id': 2003, 'label': 'Nederland (land)', 'country': 'NL', 'geonames_uri': 'http://sws.geonames.org/2750405/', 'latitude': 52.5, 'longitude': 5.75},
    {'location_id': 2004, 'label': 'België (land)', 'country': 'BE', 'geonames_uri': 'http://sws.geonames.org/2802361/', 'latitude': 50.5, 'longitude': 4.5},
    {'location_id': 2005, 'label': 'Noorwegen (land)', 'country': 'NO', 'geonames_uri': 'http://sws.geonames.org/3144096/', 'latitude': 60.472024, 'longitude': 8.468946},
    {'location_id': 2006, 'label': 'Zweden (land)', 'country': 'SE', 'geonames_uri': 'http://sws.geonames.org/2661886/', 'latitude': 60.128161, 'longitude': 18.643501},
    {'location_id': 2007, 'label': 'Duitsland (land)', 'country': 'DE', 'geonames_uri': 'http://sws.geonames.org/2921044/', 'latitude': 51.0834196, 'longitude': 10.4234469},
    {'location_id': 2008, 'label': 'Denemarken (land)', 'country': 'DK', 'geonames_uri': 'http://sws.geonames.org/2623032/', 'latitude': 56.26392, 'longitude': 9.501785},
    {'location_id': 2009, 'label': 'Finland (land)', 'country': 'FI', 'geonames_uri': 'http://sws.geonames.org/660013/', 'latitude': 61.92411, 'longitude': 25.748151},
])
Locations = pd.concat([Locations, new_locations], ignore_index=True)

# zorg dat wic_locatie een integer is
temp_locations['wic_location'] = temp_locations['wic_location'].astype('Int64')

# Reset index
Locations = Locations.reset_index(drop=True)

In [8]:
# Nu gaan we aparte tabellen maken voor entiteiten, te beginnen met personen

In [9]:
# We maken een df voor Personen en voegen notaris Henrick Schaeff handmatig toe
Persons = pd.DataFrame(columns=['person_id', 'name', 'uri', 'role', 'role_category', 'location_string', 'location_uri'])
henrick = pd.DataFrame({
    'person_id': [1],
    'name': ['Henrick Schaeff'],
    'uri': [''],
    'role': ['notary'],
    'role_category': ['notary'],
    'location_string': [''],
    'location_uri': ['']
})
Persons = pd.concat([Persons, henrick], ignore_index=True)

# df voor akten
Deeds = pd.DataFrame(columns=['deed_id', 'deed_uri', 'notary_id', 'deed_date'])

# df voor transacties
Transactions = pd.DataFrame(columns=['deed_id', 'sailor_id', 'creditor_id', 'final_debt_amount_int'])

# We voeden bovenstaande dfs met data uit de ingeladen WIC-opvarenden database
person_id_counter = Persons['person_id'].max() + 1
deed_id_counter = 1

for index, row in opvarenden_curated.iterrows():
    sailors = row['name'].split('/')
    sailor_uri = row['sailor_uri'] if pd.notna(row['sailor_uri']) else ''
    creditors = str(row['normalized_creditor_name']).split('/') if pd.notna(row['normalized_creditor_name']) else []
    deed_uri = row['deed_uri'] if pd.notna(row['deed_uri']) else ''
    sailor_role = row['normalized_role'] if pd.notna(row['normalized_role']) else ''
    sailor_role_category = row['normalized_role_category'] if pd.notna(row['normalized_role_category']) else ''
    final_debt_amount_int = row['final_debt_amount_int']
    deed_date = row['correct_deed_date'] if pd.notna(row['correct_deed_date']) else row['deed_date']
    location_string = row['location'] if pd.notna(row['location']) else ''
    location_uri = row['location_uri'] if pd.notna(row['location_uri']) else ''
    
    # Sailors gaan altijd naar Persons (geen check of ze dubbel zijn)
    for sailor in sailors:
        person = pd.DataFrame({
            'person_id': [person_id_counter],
            'name': [sailor],
            'uri': [sailor_uri if len(sailors) == 1 else ''],
            'role': [sailor_role],
            'role_category': [sailor_role_category],
            'location_string': [location_string],
            'location_uri': [location_uri]
        })
        Persons = pd.concat([Persons, person], ignore_index=True)
        sailor_id = person_id_counter
        person_id_counter += 1
    
    # Voor schuldeisers (die heel vaak voorkomen en gestandaardiseerde namen hebben) wel een check
    for creditor in creditors:
        existing_creditor = Persons[(Persons['name'] == creditor) & (Persons['role'] == 'creditor')]
        if existing_creditor.empty:
            person = pd.DataFrame({
                'person_id': [person_id_counter],
                'name': [creditor],
                'uri': [''],
                'role': ['creditor'],
                'role_category':  ['creditor'],
                'location_string': [''],
                'location_uri': ['']
            })
            Persons = pd.concat([Persons, person], ignore_index=True)
            creditor_id = person_id_counter
            person_id_counter += 1
        else:
            creditor_id = existing_creditor['person_id'].values[0]
    
    # Voeg de akte toe aan Deeds
    deed = pd.DataFrame({
        'deed_id': [deed_id_counter],
        'deed_uri': [deed_uri],
        'notary_id': [1],  # Henrick Schaeff is 1
        'deed_date': [deed_date]
    })
    Deeds = pd.concat([Deeds, deed], ignore_index=True)
    deed_id = deed_id_counter
    deed_id_counter += 1
    
    # Transacties tussen opvarenden en schuldeisers gaan naar Transactions
    for sailor in sailors:
        for creditor in creditors:
            transaction = pd.DataFrame({
                'deed_id': [deed_id],
                'sailor_id': [sailor_id],
                'creditor_id': [creditor_id],
                'final_debt_amount_int': [final_debt_amount_int]
            })
            Transactions = pd.concat([Transactions, transaction], ignore_index=True)

In [10]:
# Voeg de locaties toe aan de Persons DataFrame
Persons['location_standardized'] = None

# Dict om deed_id naar deed_uri te mappen voor snelle toegang
deed_id_to_uri = Deeds.set_index('deed_id')['deed_uri'].to_dict()

# We itereren over iedere opvarende in Persons ...
for idx, person in Persons[~Persons['role'].isin(['notary', 'creditor'])].iterrows():
    sailor_id = person['person_id']
    # ... zoeken naar de gekoppelde transacties ...
    sailor_transactions = Transactions[Transactions['sailor_id'] == sailor_id]
    for _, transaction in sailor_transactions.iterrows():
        deed_id = transaction['deed_id']
        deed_uri = deed_id_to_uri.get(deed_id, None)
        if deed_uri:
            # ... en zoeken naar de locatie van de schuldeiser
            temp_location_matches = temp_locations[(temp_locations['deed_uri'] == deed_uri) & (temp_locations['name'] == person['name'])]
            if not temp_location_matches.empty:
                # als er meerdere matches zijn, kies de eerste
                wic_location = temp_location_matches.iloc[0]['wic_location']
                country_code = temp_location_matches.iloc[0]['country_code']
                if pd.notna(wic_location):
                    Persons.at[idx, 'location_standardized'] = wic_location
                    break # we hebben een match gevonden, dus we kunnen stoppen

                # Tijdelijke (?) oplossing voor locaties zonder Geonames koppeling
                if pd.isna(wic_location) & pd.notna(country_code):
                    if country_code == 'FR':
                        Persons.at[idx, 'location_standardized'] = 2000
                    elif country_code == 'IE':
                        Persons.at[idx, 'location_standardized'] = 2001
                    elif country_code == 'GB':
                        Persons.at[idx, 'location_standardized'] = 2002
                    elif country_code == 'NL':
                        Persons.at[idx, 'location_standardized'] = 2003
                    elif country_code == 'BE':
                        Persons.at[idx, 'location_standardized'] = 2004
                    elif country_code == 'NO':
                        Persons.at[idx, 'location_standardized'] = 2005
                    elif country_code == 'SE':
                        Persons.at[idx, 'location_standardized'] = 2006
                    elif country_code == 'DE':
                        Persons.at[idx, 'location_standardized'] = 2007
                    elif country_code == 'DK':
                        Persons.at[idx, 'location_standardized'] = 2008
                    elif country_code == 'FI':
                        Persons.at[idx, 'location_standardized'] = 2009

In [11]:
# Nu gaan we individuele reizen van schepen in een aparte tabel zetten

In [12]:
# We werken met oude datums, dus deed date moet een string zijn
opvarenden_curated['deed_date'] = opvarenden_curated['deed_date'].astype(str)

# Initialiseer counter voor Voyages
global_voyage_id = 0

# Functie om afzonderlijke reizen te detecteren
def detect_voyages(group):
    global global_voyage_id
    group = group.sort_values('deed_date').reset_index(drop=True)
    if pd.notna(group['normalized_ship_name'].iloc[0]):  # Is er een genormaliseerde schipnaam?
        group['deed_date_period'] = group['deed_date'].apply(lambda x: pd.Period(x, freq='D'))
        group['voyage_id'] = group['deed_date_period'].diff().apply(lambda x: x.n > 180 if pd.notna(x) else False).cumsum() # meer dan 180 dagen verschil? nieuwe reis!
        group['voyage_id'] += global_voyage_id
        global_voyage_id = group['voyage_id'].max() + 1  # id ophogen
    else:
        group['voyage_id'] = pd.NA  # NA als er geen genormaliseerde schipnaam is
    return group

# We voeren deze functie uit voor iedere combinatie van schip en organisatie
opvarenden_curated_updated = opvarenden_curated.groupby(['normalized_ship_name', 'organization']).apply(detect_voyages).reset_index(drop=True)

# Aanmaken Voyages df
Voyages = opvarenden_curated_updated.groupby('voyage_id').agg(
    ship_name=('normalized_ship_name', 'first'),
    organization=('imp_organization', 'first'),
    first_deed_date=('deed_date', 'first'),
    last_deed_date=('deed_date', 'last')
).reset_index()

# Updaten opvarenden_curated met nieuwe informatie
opvarenden_curated = opvarenden_curated_updated


In [13]:
# Nu voegen we de Voyage_ids to aan Transactions
Transactions['voyage_id'] = None

# Dict om deed_id naar uri te mappen voor snelle lookup
deed_id_to_uri = pd.Series(Deeds.deed_uri.values, index=Deeds.deed_id).to_dict()

# Dict omt uri naar voayge id te mappen voor snelle lookup
deed_uri_to_voyage_id = pd.Series(opvarenden_curated_updated.voyage_id.values, index=opvarenden_curated_updated.deed_uri).to_dict()

# Update voyage_id in Transactions
for index, row in Transactions.iterrows():
    deed_id = row['deed_id']
    if deed_id in deed_id_to_uri:
        deed_uri = deed_id_to_uri[deed_id]
        if deed_uri in deed_uri_to_voyage_id:
            Transactions.at[index, 'voyage_id'] = deed_uri_to_voyage_id[deed_uri]

In [14]:
# Zorg dat location_standardized een integer is
Persons['location_standardized'] = Persons['location_standardized'].astype('Int64')

Nu hebben we alle afzonderlijke dataframes. Tijd om een SQLite-database te maken!

In [15]:
import sqlite3

conn = sqlite3.connect('wic-opvarenden.db')
cursor = conn.cursor()

# Drop tables als ze al bestaan
cursor.execute('DROP TABLE IF EXISTS Deeds')
cursor.execute('DROP TABLE IF EXISTS Locations')
cursor.execute('DROP TABLE IF EXISTS Persons')
cursor.execute('DROP TABLE IF EXISTS Transactions')
cursor.execute('DROP TABLE IF EXISTS Voyages')

# Maak benodigde tables aan (inclusief foreign key-relaties)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Deeds (
        deed_id INTEGER PRIMARY KEY,
        deed_uri TEXT,
        notary_id INTEGER,
        deed_date TEXT,
        FOREIGN KEY (notary_id) REFERENCES Persons(person_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Locations (
        location_id INTEGER PRIMARY KEY,
        label TEXT,
        country TEXT,
        geonames_uri TEXT,
        latitude REAL,
        longitude REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Persons (
        person_id INTEGER PRIMARY KEY,
        name TEXT,
        uri TEXT,
        role TEXT,
        role_category TEXT,
        location_string TEXT,
        location_uri TEXT,
        location_standardized INTEGER,
        FOREIGN KEY (location_standardized) REFERENCES Locations(location_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Transactions (
        deed_id INTEGER,
        sailor_id INTEGER,
        creditor_id INTEGER,
        final_debt_amount_int INTEGER,
        voyage_id INTEGER,
        FOREIGN KEY (deed_id) REFERENCES Deeds(deed_id),
        FOREIGN KEY (sailor_id) REFERENCES Persons(person_id),
        FOREIGN KEY (creditor_id) REFERENCES Persons(person_id),
        FOREIGN KEY (voyage_id) REFERENCES Voyages(voyage_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Voyages (
        voyage_id INTEGER PRIMARY KEY,
        ship_name TEXT,
        organization TEXT,
        first_deed_date TEXT,
        last_deed_date TEXT
    )
''')

# Dat uit df inlezen in de database
Deeds.to_sql('Deeds', conn, if_exists='append', index=False)
Locations.to_sql('Locations', conn, if_exists='append', index=False)
Persons.to_sql('Persons', conn, if_exists='append', index=False)
Transactions.to_sql('Transactions', conn, if_exists='append', index=False)
Voyages.to_sql('Voyages', conn, if_exists='append', index=False)

# Commit en close
conn.commit()
conn.close()

In [16]:
import sqlite3

conn = sqlite3.connect('wic-opvarenden.db')
cursor = conn.cursor()

# Indexen voor optimalisatie van queries
with conn:

    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_persons_location_standardized 
        ON Persons (location_standardized);
    """)
    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_persons_role_category 
        ON Persons (role_category);
    """)


    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_transactions_sailor_id 
        ON Transactions (sailor_id);
    """)
    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_transactions_voyage_id 
        ON Transactions (voyage_id);
    """)
    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_transactions_deed_id
        ON Transactions (deed_id);
    """)


    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_deeds_deed_date 
        ON Deeds (deed_date);
    """)


    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_voyages_first_deed_date 
        ON Voyages (first_deed_date);
    """)
    conn.execute("""
        CREATE INDEX IF NOT EXISTS idx_voyages_last_deed_date 
        ON Voyages (last_deed_date);
    """)

conn.close()



In [17]:
import pandas as pd
import sqlite3
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from datetime import datetime

queries = {
    'Top voyages': """
SELECT V.voyage_id, 
       V.ship_name || ' (' || V.organization || ', ' || strftime('%Y', V.last_deed_date) || ')' AS voyage,
       CAST(ROUND(SUM(T.final_debt_amount_int)) AS INTEGER) AS total_debt, 
       CAST(ROUND(AVG(T.final_debt_amount_int)) AS INTEGER) AS average_debt, 
       COUNT(T.deed_id) AS number_of_transactions,
       strftime('%Y', V.last_deed_date) AS voyage_year
FROM Voyages V
JOIN Transactions T ON V.voyage_id = T.voyage_id
GROUP BY V.voyage_id, V.ship_name, V.organization, voyage_year
ORDER BY total_debt DESC;
""",
    'Top creditors': """
SELECT P.name AS creditor_name, SUM(T.final_debt_amount_int) AS total_amount_lent
FROM Transactions T
JOIN Persons P ON T.creditor_id = P.person_id
GROUP BY P.name
ORDER BY total_amount_lent DESC;
""",
    'Nationality per voyage': """
SELECT V.voyage_id, 
       V.ship_name || ' (' || V.organization || ', ' || strftime('%Y', V.last_deed_date) || ')' AS voyage,
       COUNT(CASE WHEN L.country = 'NL' THEN T.deed_id END) AS NL,
       COUNT(CASE WHEN L.country = 'DE' THEN T.deed_id END) AS DE,
       COUNT(CASE WHEN L.country = 'BE' THEN T.deed_id END) AS BE,
       COUNT(CASE WHEN L.country = 'GB' THEN T.deed_id END) AS GB,
       COUNT(CASE WHEN L.country = 'SE' THEN T.deed_id END) AS SE,
       COUNT(CASE WHEN L.country = 'NO' THEN T.deed_id END) AS NO,
       COUNT(CASE WHEN L.country = 'DK' THEN T.deed_id END) AS DK,
       COUNT(CASE WHEN L.country = 'FR' THEN T.deed_id END) AS FR,
       COUNT(CASE WHEN L.country NOT IN ('NL', 'DE', 'BE', 'GB', 'SE', 'NO', 'DK', 'FR') AND L.country IS NOT NULL THEN T.deed_id END) AS other,
       COUNT(CASE WHEN L.country IS NULL OR S.location_standardized IS NULL THEN T.deed_id END) AS unknown,
       COUNT(T.deed_id) AS total
FROM Voyages V
JOIN Transactions T ON V.voyage_id = T.voyage_id
JOIN Persons S ON T.sailor_id = S.person_id
LEFT JOIN Locations L ON S.location_standardized = L.location_id
GROUP BY V.voyage_id, voyage
ORDER BY total DESC;
""",
    'Amount lent': """
SELECT L.country, 
       CAST(SUM(T.final_debt_amount_int) AS INTEGER) AS total_amount_lent, 
       COUNT(T.deed_id) AS number_of_loans,
       CAST(ROUND(AVG(T.final_debt_amount_int)) AS INTEGER) AS average_loan_amount
FROM Transactions T
JOIN Persons S ON T.sailor_id = S.person_id
JOIN Locations L ON S.location_standardized = L.location_id
WHERE S.role_category = 'sailor'
GROUP BY L.country
ORDER BY total_amount_lent DESC;
""",
    'Nationality per creditor': """
SELECT P.name AS creditor_name, 
       COUNT(CASE WHEN L.country = 'NL' THEN T.deed_id END) AS NL,
       COUNT(CASE WHEN L.country = 'DE' THEN T.deed_id END) AS DE,
       COUNT(CASE WHEN L.country = 'BE' THEN T.deed_id END) AS BE,
       COUNT(CASE WHEN L.country = 'GB' THEN T.deed_id END) AS GB,
       COUNT(CASE WHEN L.country = 'SE' THEN T.deed_id END) AS SE,
       COUNT(CASE WHEN L.country = 'NO' THEN T.deed_id END) AS NO,
       COUNT(CASE WHEN L.country = 'DK' THEN T.deed_id END) AS DK,
       COUNT(CASE WHEN L.country = 'FR' THEN T.deed_id END) AS FR,
       COUNT(CASE WHEN L.country NOT IN ('NL', 'DE', 'BE', 'GB', 'SE', 'NO', 'DK', 'FR') AND L.country IS NOT NULL THEN T.deed_id END) AS other,
       COUNT(CASE WHEN L.country IS NULL OR S.location_standardized IS NULL THEN T.deed_id END) AS unknown,
       COUNT(T.deed_id) AS total
FROM Transactions T
JOIN Persons P ON T.creditor_id = P.person_id
JOIN Persons S ON T.sailor_id = S.person_id
LEFT JOIN Locations L ON S.location_standardized = L.location_id
GROUP BY P.name
ORDER BY total DESC;
""",
    'Role per creditor': """
WITH RoleCounts AS (
    SELECT 
        P.name AS creditor_name,
        S.role,
        COUNT(T.deed_id) AS transaction_count
    FROM Transactions T
    JOIN Persons P ON T.creditor_id = P.person_id
    JOIN Persons S ON T.sailor_id = S.person_id
    WHERE S.role IS NOT NULL
    GROUP BY P.name, S.role
),
TotalCounts AS (
    SELECT 
        creditor_name, 
        SUM(transaction_count) AS total_transactions
    FROM RoleCounts
    GROUP BY creditor_name
)
SELECT 
    RC.creditor_name,
    MAX(CASE WHEN RC.role = 'adelborst' THEN RC.transaction_count ELSE 0 END) AS adelborst,
    MAX(CASE WHEN RC.role = 'appoincte' THEN RC.transaction_count ELSE 0 END) AS appoincte,
    MAX(CASE WHEN RC.role = 'assistent' THEN RC.transaction_count ELSE 0 END) AS assistent,
    MAX(CASE WHEN RC.role = 'barbier' THEN RC.transaction_count ELSE 0 END) AS barbier,
    MAX(CASE WHEN RC.role = 'boekhouder' THEN RC.transaction_count ELSE 0 END) AS boekhouder,
    MAX(CASE WHEN RC.role = 'bootsgezel' THEN RC.transaction_count ELSE 0 END) AS bootsgezel,
    MAX(CASE WHEN RC.role = 'bosschieter' THEN RC.transaction_count ELSE 0 END) AS bosschieter,
    MAX(CASE WHEN RC.role = 'bottelier' THEN RC.transaction_count ELSE 0 END) AS bottelier,
    MAX(CASE WHEN RC.role = 'botteliersjongen' THEN RC.transaction_count ELSE 0 END) AS botteliersjongen,
    MAX(CASE WHEN RC.role = 'botteliersmaat' THEN RC.transaction_count ELSE 0 END) AS botteliersmaat,
    MAX(CASE WHEN RC.role = 'chirurgijn' THEN RC.transaction_count ELSE 0 END) AS chirurgijn,
    MAX(CASE WHEN RC.role = 'chirurgijnsjongen' THEN RC.transaction_count ELSE 0 END) AS chirurgijnsjongen,
    MAX(CASE WHEN RC.role = 'commandeur' THEN RC.transaction_count ELSE 0 END) AS commandeur,
    MAX(CASE WHEN RC.role = 'commies' THEN RC.transaction_count ELSE 0 END) AS commies,
    MAX(CASE WHEN RC.role = 'commissaris' THEN RC.transaction_count ELSE 0 END) AS commissaris,
    MAX(CASE WHEN RC.role = 'creditor' THEN RC.transaction_count ELSE 0 END) AS creditor,
    MAX(CASE WHEN RC.role = 'derde stuurman' THEN RC.transaction_count ELSE 0 END) AS derde_stuurman,
    MAX(CASE WHEN RC.role = 'hoogbootsman' THEN RC.transaction_count ELSE 0 END) AS hoogbootsman,
    MAX(CASE WHEN RC.role = 'hooploper' THEN RC.transaction_count ELSE 0 END) AS hooploper,
    MAX(CASE WHEN RC.role = 'jongen' THEN RC.transaction_count ELSE 0 END) AS jongen,
    MAX(CASE WHEN RC.role = 'kajuitwachter' THEN RC.transaction_count ELSE 0 END) AS kajuitwachter,
    MAX(CASE WHEN RC.role = 'kapitein' THEN RC.transaction_count ELSE 0 END) AS kapitein,
    MAX(CASE WHEN RC.role = 'kapitein-luitenant' THEN RC.transaction_count ELSE 0 END) AS kapitein_luitenant,
    MAX(CASE WHEN RC.role = 'klerk' THEN RC.transaction_count ELSE 0 END) AS klerk,
    MAX(CASE WHEN RC.role = 'kok' THEN RC.transaction_count ELSE 0 END) AS kok,
    MAX(CASE WHEN RC.role = 'koksmaat' THEN RC.transaction_count ELSE 0 END) AS koksmaat,
    MAX(CASE WHEN RC.role = 'konstabel' THEN RC.transaction_count ELSE 0 END) AS konstabel,
    MAX(CASE WHEN RC.role = 'konstabelsmaat' THEN RC.transaction_count ELSE 0 END) AS konstabelsmaat,
    MAX(CASE WHEN RC.role = 'koperslager' THEN RC.transaction_count ELSE 0 END) AS koperslager,
    MAX(CASE WHEN RC.role = 'korporaal' THEN RC.transaction_count ELSE 0 END) AS korporaal,
    MAX(CASE WHEN RC.role = 'kuiper' THEN RC.transaction_count ELSE 0 END) AS kuiper,
    MAX(CASE WHEN RC.role = 'kuipersmaat' THEN RC.transaction_count ELSE 0 END) AS kuipersmaat,
    MAX(CASE WHEN RC.role = 'kwartiermeester' THEN RC.transaction_count ELSE 0 END) AS kwartiermeester,
    MAX(CASE WHEN RC.role = 'lansmissaat' THEN RC.transaction_count ELSE 0 END) AS lansmissaat,
    MAX(CASE WHEN RC.role = 'luitenant' THEN RC.transaction_count ELSE 0 END) AS luitenant,
    MAX(CASE WHEN RC.role = 'mastklimmer' THEN RC.transaction_count ELSE 0 END) AS mastklimmer,
    MAX(CASE WHEN RC.role = 'matroos' THEN RC.transaction_count ELSE 0 END) AS matroos,
    MAX(CASE WHEN RC.role = 'metselaar' THEN RC.transaction_count ELSE 0 END) AS metselaar,
    MAX(CASE WHEN RC.role = 'notary' THEN RC.transaction_count ELSE 0 END) AS notary,
    MAX(CASE WHEN RC.role = 'onderbarbier' THEN RC.transaction_count ELSE 0 END) AS onderbarbier,
    MAX(CASE WHEN RC.role = 'onderchirurgijn' THEN RC.transaction_count ELSE 0 END) AS onderchirurgijn,
    MAX(CASE WHEN RC.role = 'ondercommies' THEN RC.transaction_count ELSE 0 END) AS ondercommies,
    MAX(CASE WHEN RC.role = 'onderkoopman' THEN RC.transaction_count ELSE 0 END) AS onderkoopman,
    MAX(CASE WHEN RC.role = 'onderkorporaal' THEN RC.transaction_count ELSE 0 END) AS onderkorporaal,
    MAX(CASE WHEN RC.role = 'onderkuiper' THEN RC.transaction_count ELSE 0 END) AS onderkuiper,
    MAX(CASE WHEN RC.role = 'onderstuurman' THEN RC.transaction_count ELSE 0 END) AS onderstuurman,
    MAX(CASE WHEN RC.role = 'ondertimmerman' THEN RC.transaction_count ELSE 0 END) AS ondertimmerman,
    MAX(CASE WHEN RC.role = 'ondertrompetter' THEN RC.transaction_count ELSE 0 END) AS ondertrompetter,
    MAX(CASE WHEN RC.role = 'onderzeilmaker' THEN RC.transaction_count ELSE 0 END) AS onderzeilmaker,
    MAX(CASE WHEN RC.role = 'opperbarbier' THEN RC.transaction_count ELSE 0 END) AS opperbarbier,
    MAX(CASE WHEN RC.role = 'opperchirurgijn' THEN RC.transaction_count ELSE 0 END) AS opperchirurgijn,
    MAX(CASE WHEN RC.role = 'oppercommies' THEN RC.transaction_count ELSE 0 END) AS oppercommies,
    MAX(CASE WHEN RC.role = 'opperkuiper' THEN RC.transaction_count ELSE 0 END) AS opperkuiper,
    MAX(CASE WHEN RC.role = 'opperstuurman' THEN RC.transaction_count ELSE 0 END) AS opperstuurman,
    MAX(CASE WHEN RC.role = 'oppertimmerman' THEN RC.transaction_count ELSE 0 END) AS oppertimmerman,
    MAX(CASE WHEN RC.role = 'opperzeilmaker' THEN RC.transaction_count ELSE 0 END) AS opperzeilmaker,
    MAX(CASE WHEN RC.role = 'provoost' THEN RC.transaction_count ELSE 0 END) AS provoost,
    MAX(CASE WHEN RC.role = 'rector' THEN RC.transaction_count ELSE 0 END) AS rector,
    MAX(CASE WHEN RC.role = 'roermaker' THEN RC.transaction_count ELSE 0 END) AS roermaker,
    MAX(CASE WHEN RC.role = 'scheepsjongen' THEN RC.transaction_count ELSE 0 END) AS scheepsjongen,
    MAX(CASE WHEN RC.role = 'scheepskorporaal' THEN RC.transaction_count ELSE 0 END) AS scheepskorporaal,
    MAX(CASE WHEN RC.role = 'scherprechter' THEN RC.transaction_count ELSE 0 END) AS scherprechter,
    MAX(CASE WHEN RC.role = 'schieman' THEN RC.transaction_count ELSE 0 END) AS schieman,
    MAX(CASE WHEN RC.role = 'schiemansmaat' THEN RC.transaction_count ELSE 0 END) AS schiemansmaat,
    MAX(CASE WHEN RC.role = 'schipper' THEN RC.transaction_count ELSE 0 END) AS schipper,
    MAX(CASE WHEN RC.role = 'schoolmeester' THEN RC.transaction_count ELSE 0 END) AS schoolmeester,
    MAX(CASE WHEN RC.role = 'schrijver' THEN RC.transaction_count ELSE 0 END) AS schrijver,
    MAX(CASE WHEN RC.role = 'secretaris' THEN RC.transaction_count ELSE 0 END) AS secretaris,
    MAX(CASE WHEN RC.role = 'sergeant' THEN RC.transaction_count ELSE 0 END) AS sergeant,
    MAX(CASE WHEN RC.role = 'slotenmaker' THEN RC.transaction_count ELSE 0 END) AS slotenmaker,
    MAX(CASE WHEN RC.role = 'smit' THEN RC.transaction_count ELSE 0 END) AS smit,
    MAX(CASE WHEN RC.role = 'sobra cargo' THEN RC.transaction_count ELSE 0 END) AS sobra_cargo,
    MAX(CASE WHEN RC.role = 'sobro cargo' THEN RC.transaction_count ELSE 0 END) AS sobro_cargo,
    MAX(CASE WHEN RC.role = 'soldaat' THEN RC.transaction_count ELSE 0 END) AS soldaat,
    MAX(CASE WHEN RC.role = 'spekdrager' THEN RC.transaction_count ELSE 0 END) AS spekdrager,
    MAX(CASE WHEN RC.role = 'speksnijder' THEN RC.transaction_count ELSE 0 END) AS speksnijder,
    MAX(CASE WHEN RC.role = 'steenhouwer' THEN RC.transaction_count ELSE 0 END) AS steenhouwer,
    MAX(CASE WHEN RC.role = 'stuurman' THEN RC.transaction_count ELSE 0 END) AS stuurman,
    MAX(CASE WHEN RC.role = 'tamboer' THEN RC.transaction_count ELSE 0 END) AS tamboer,
    MAX(CASE WHEN RC.role = 'timmerman' THEN RC.transaction_count ELSE 0 END) AS timmerman,
    MAX(CASE WHEN RC.role = 'trompetter' THEN RC.transaction_count ELSE 0 END) AS trompetter,
    MAX(CASE WHEN RC.role = 'vaandrig' THEN RC.transaction_count ELSE 0 END) AS vaandrig,
    MAX(CASE WHEN RC.role = 'varensgezel' THEN RC.transaction_count ELSE 0 END) AS varensgezel,
    MAX(CASE WHEN RC.role = 'varensman' THEN RC.transaction_count ELSE 0 END) AS varensman,
    MAX(CASE WHEN RC.role = 'veldpijper' THEN RC.transaction_count ELSE 0 END) AS veldpijper,
    MAX(CASE WHEN RC.role = 'voorlezer' THEN RC.transaction_count ELSE 0 END) AS voorlezer,
    MAX(CASE WHEN RC.role = 'zeilmaker' THEN RC.transaction_count ELSE 0 END) AS zeilmaker,
    MAX(CASE WHEN RC.role = 'ziekentrooster' THEN RC.transaction_count ELSE 0 END) AS ziekentrooster,
    MAX(CASE WHEN RC.role = 'zwaardveger' THEN RC.transaction_count ELSE 0 END) AS zwaardveger,
    MAX(CASE WHEN RC.role = '' THEN RC.transaction_count ELSE 0 END) AS [geen rol],
    TC.total_transactions
FROM RoleCounts RC
JOIN TotalCounts TC 
    ON RC.creditor_name = TC.creditor_name
GROUP BY RC.creditor_name, TC.total_transactions
ORDER BY TC.total_transactions DESC;
""",
    'Role category per creditor': """
SELECT 
    P.name AS creditor_name, 
    COUNT(CASE WHEN S.role_category = 'sailor' THEN T.deed_id END) AS sailor_transactions,
    COUNT(CASE WHEN S.role_category = 'soldier' THEN T.deed_id END) AS soldier_transactions,
    COUNT(CASE WHEN S.role_category IS NULL OR S.role_category NOT IN ('sailor', 'soldier') THEN T.deed_id END) AS unknown_transactions,
    COUNT(T.deed_id) AS total_transactions
FROM 
    Transactions T
JOIN 
    Persons P ON T.creditor_id = P.person_id
JOIN 
    Persons S ON T.sailor_id = S.person_id
GROUP BY 
    P.name
ORDER BY 
    total_transactions DESC;
""",
    'Transactions per role': """
SELECT 
    P.role, 
    COUNT(T.deed_id) AS transaction_count,
    CAST(ROUND(AVG(T.final_debt_amount_int)) AS INTEGER) AS average_debt_per_transaction,
    CAST(SUM(T.final_debt_amount_int) AS INTEGER) AS total_debt_amount
FROM Persons P
JOIN Transactions T ON P.person_id = T.sailor_id
WHERE P.role IS NOT NULL
GROUP BY P.role
ORDER BY total_debt_amount DESC;
""",
    'Top 20 debt amounts': """
WITH TopAmounts AS (
    SELECT 
        final_debt_amount_int
    FROM Transactions
    GROUP BY final_debt_amount_int
    ORDER BY COUNT(*) DESC
    LIMIT 20
)
SELECT 
    T.final_debt_amount_int AS debt_amount,
    P.role AS sailor_role,
    COUNT(*) AS frequency
FROM Transactions T
JOIN Persons P
  ON T.sailor_id = P.person_id
JOIN TopAmounts TA
  ON T.final_debt_amount_int = TA.final_debt_amount_int
GROUP BY T.final_debt_amount_int, P.role
ORDER BY T.final_debt_amount_int, frequency DESC;

""",
    'Five women': """
SELECT
    C.name AS creditor_name,
    D.deed_date,
    S.name AS borrower_name,
    S.role AS borrower_role,
    L.label AS borrower_location_label,
    L.country AS borrower_location_country,
    T.final_debt_amount_int AS debt_amount
FROM Transactions T
JOIN Persons C
  ON T.creditor_id = C.person_id
JOIN Persons S
  ON T.sailor_id = S.person_id
JOIN Deeds D
  ON T.deed_id = D.deed_id
LEFT JOIN Locations L
  ON S.location_standardized = L.location_id
WHERE C.name IN (
    'Cathalina d'' Accre',
    'Annetgen Boeijensdr.',
    'Annetgen Reijnderts',
    'Anna Jansdr.',
    'Stijntgen Adriaens',
    'Grietgen Henricxdr.'
)
ORDER BY 
    C.name,
    D.deed_date;
"""
}

excel_filename = f"{datetime.now().strftime('%Y-%m-%d')}-wic_opvarenden.xlsx"

with sqlite3.connect('wic-opvarenden.db') as conn:
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        for sheet_name, query in queries.items():

            df = pd.read_sql_query(query, conn)
            
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            
            worksheet = writer.sheets[sheet_name]

            header_font = Font(bold=True)
            for col in range(1, len(df.columns) + 1):
                cell = worksheet.cell(row=1, column=col)
                cell.font = header_font

            worksheet.auto_filter.ref = worksheet.dimensions
            
            # Breedte van de kolommen aanpassen
            for col in worksheet.columns:
                max_length = 0
                column_letter = col[0].column_letter
                for cell in col:
                    cell_value = cell.value
                    if cell_value is not None:
                        max_length = max(max_length, len(str(cell_value)))

                adjusted_width = max_length + 2
                worksheet.column_dimensions[column_letter].width = adjusted_width